In [1]:
import pandas as pd;
import numpy as np;
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [2]:
from bs4 import BeautifulSoup
import requests

<p>Use BeautifulSoup to extract table and populate all records (between td tag) into an array (named data).</p>

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')
    
data = []
table = soup.find('tbody')
rows = table.find_all('tr')

for row in rows:
    cols = row.find_all('td')
    data.append([col.text for col in cols])

<p>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.</p>

In [4]:
df = pd.DataFrame(data,columns=['PostalCode','Borough','Neighborhood'])

<p>Only process the cells that have an assigned borough.
Ignore cells with a borough that is Not assigned.</p>

In [5]:
df = df[df['Borough'].notnull()]
df = df[~df['Borough'].str.contains('Not assigned')]

<p>More than one neighborhood can exist in one postal code area.</p>

In [6]:
df['Neighborhood'] = df['Neighborhood'].str.strip()
df=df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

<p>If a cell has a borough but a Not assigned neighborhood,
then the neighborhood will be the same as the borough.</p>

In [7]:
df['Neighborhood'] = np.where(df['Neighborhood'].str.contains('Not assigned'),df['Borough'],df['Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<p>Show the number of rows.</p>

In [8]:
df.shape

(103, 3)

<p>Retrieve geographical coordinates of postal code</p>

In [9]:
geo_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df1=pd.merge(df,geo_df,on='PostalCode')
df1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<p>Show map of Toronto using folium</p>

In [10]:
import folium
import json

map = folium.Map(location=(43.653908, -79.384293), zoom_start=12)
  
display(map)

<p>Append neighborhood clusters to Toronto map</p>

In [11]:
for index, row in df1.iterrows():
    label = '{}, {}'.format(row['Neighborhood'], row['Borough'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius=3,
        popup=label,
        color='red',
        fill=True).add_to(map) 

display(map)